In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from torch.utils.data import Dataset, DataLoader

np.random.seed(1234)

In [2]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # GPU name

# CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using device: {device}")

NVIDIA GeForce GTX 1650
Using device: cuda


In [3]:
class DNN(nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        modules = []
        for i in range(len(layers) - 2):  # Exclude last layer for activation
            modules.append(nn.Linear(layers[i], layers[i+1]))
            modules.append(nn.Tanh())
        modules.append(nn.Linear(layers[-2], layers[-1]))  # Last layer (no activation)
        self.network = nn.Sequential(*modules)

        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.network(x)

In [8]:
def create_dataset(config):
    
    p = config["points"]
    lbc = config["lbc"]
    rbc = config["rbc"]
    total = p[0] + 2*p[1]
    xl = torch.zeros(p[1])
    xr = torch.ones(p[1])
    x = torch.rand(p[0])
    x = torch.hstack((xl, xr, x))
    x = torch.sort(x)[0].view(-1, 1)
    lbc = torch.tensor(lbc, dtype=torch.float).repeat(total, 1)
    rbc = torch.tensor(rbc, dtype=torch.float).repeat(total, 1)

    data = torch.cat((x, lbc, rbc), dim=1).requires_grad_().to(device)
    return data

In [10]:
# Fixed = 1
# Pinned = 2
# Free = 3

p = [100, 10]
fixed = 1
pinned = 2
free = 3
beam1 = {"points": p, "lbc": fixed, "rbc": free}

data = create_dataset(beam1)
print(data.shape)


torch.Size([120, 3])


In [ ]:
class PINN():
    def __init__(self, data, layers):

        self.data = data
        self.dnn = DNN(layers).to(device)

        self.optimizer_adam = torch.optim.Adam(self.dnn.parameters(), lr=0.005)

    def model_value(self, data):
        out = self.dnn(data)
        u = out[:, 0:1]
        m = out[:, 1:2]
        return u, m

    def boundary_condition(self, mask, data, u, u_x, m, m_x, left=True):

        if left:
            bc_data = data[mask, 1:4]
        else:
            bc_data = data[mask, 4:7]
        bc_mask = bc_data.bool()

        u_bc, u_x_bc, m_bc, m_x_bc = u[mask], u_x[mask], m[mask], m_x[mask]
        # Initialize losses
        bc_fixed_loss = torch.tensor(0.0, device=device)
        bc_pinned_loss = torch.tensor(0.0, device=device)
        bc_free_loss = torch.tensor(0.0, device=device)

        # Fixed BC
        if True in bc_mask[:, 0]:
            bc_fixed_loss = torch.mean(torch.pow(u_bc[bc_mask[:, 0]], 2))
            bc_fixed_loss += torch.mean(torch.pow(u_x_bc[bc_mask[:, 0]], 2))

        # Pinned BC
        if True in bc_mask[:, 1]:
            bc_pinned_loss = torch.mean(torch.pow(u_bc[bc_mask[:, 1]], 2))
            bc_pinned_loss += torch.mean(torch.pow(m_bc[bc_mask[:, 1]], 2))

        # Free BC
        if True in bc_mask[:, 2]:
            bc_free_loss = torch.mean(torch.pow(m_bc[bc_mask[:, 2]], 2))
            bc_free_loss += torch.mean(torch.pow(m_x_bc[bc_mask[:, 2]], 2))

        total_loss = bc_fixed_loss + bc_pinned_loss + bc_free_loss
        return total_loss


    def loss_func(self, data):
        u, m = self.model_value(data)

        mask_left = (data[:, 0] == 0)
        mask_right = (data[:, 0] == 1)

        u_x = torch.autograd.grad(u, data, torch.ones_like(u), create_graph=True)[0][:, 0:1]
        m_x = torch.autograd.grad(m, data, torch.ones_like(m), create_graph=True)[0][:, 0:1]

        if True in mask_left:
            bc_left_loss = self.boundary_condition(mask_left, data, u, u_x, m, m_x)
        else:
            bc_left_loss = torch.zeros(1, device=device)

        if True in mask_right:
            bc_right_loss = self.boundary_condition(mask_right, data, u, u_x, m, m_x, False)
        else:
            bc_right_loss = torch.zeros(1, device=device)

        return bc_left_loss, bc_right_loss
        


    def train(self, epochs=1000):
        self.dnn.train()

        for epoch in range(epochs):
            total_loss = 0
            for data_batch in self.dataloader:
                pde_loss, bc_left_loss, bc_right_loss = self.loss_func(data_batch)
                loss = pde_loss + bc_left_loss + bc_right_loss
                
                self.optimizer_adam.zero_grad()
                loss.backward()
                self.optimizer_adam.step()
                total_loss += loss.item()

            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Loss = {total_loss:.6f}")
                